In [24]:
##加载模型
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
from tqdm import tqdm
tokenizer = AutoTokenizer.from_pretrained("detected_model")
model = AutoModelForCausalLM.from_pretrained("detected_model", torch_dtype=torch.bfloat16)

In [25]:
#加载数据集
import os
os.environ['TOKENIZERS_PARALLELISM'] = "True"
flatten = lambda l : [x for s in l for x in s]
shuffle = lambda l : random.sample(l, k=len(l))
data_path='dataset/valid.json'
print("loading from json...")
with open(data_path, 'r') as f:
    data = f.read()
    lines=json.loads(data)
tokens=[d['text'] for d in lines]

loading from json...


In [22]:
#计算困惑度
model.eval()
token=tokens[0]
token_len=len(token)
input_ids=tokenizer.encode(token,return_tensors='pt')
#print(input_ids)
nlls=[]#存储每个片段的负对数似然值
prev_end_loc=0#上一个片段的结束位置
stride=30#这个地方应该可以调的
max_length=40#这个地方也可以调
for begin_loc in tqdm(range(0,token_len,stride)):
    end_loc=min(begin_loc+max_length,token_len)
    if end_loc==token_len:
        break
    trg_len=end_loc-prev_end_loc
    input=input_ids[:,begin_loc:end_loc]
    with torch.no_grad():
        outputs=model(input_ids,labels=input_ids)
        log_likelihood=outputs.loss
    nlls.append(log_likelihood)
    prev_end_loc=end_loc
ppl=torch.exp(torch.stack(nlls).mean())
print("perplexity:",ppl)

  0%|          | 0/130 [00:00<?, ?it/s]

 99%|█████████▉| 129/130 [1:00:13<00:28, 28.01s/it]

perplexity: tensor(8.6250, dtype=torch.bfloat16)


In [40]:
sum_perplexity=[]
for i in range(tokens):
    # 输入句子
    sentence = tokens[-8]

    # 将句子分词并转换为模型的输入格式
    input_ids = tokenizer.encode(sentence, return_tensors='pt')

    # 使用模型预测每个词的概率
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss

    # 计算困惑度
    perplexity = torch.exp(loss)
     
    sum_perplexity.append(perplexity.item())

with open('perplexity.json', 'w') as f:
    json.dump(sum_perplexity, f)



Perplexity: 7.75
